In [31]:
import random
from IPython.display import clear_output
import time

### Global Variables

In [33]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
numerical_ranks = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

### Card Class 

This class defines the blueprint of a single card.

In [35]:
class Card:

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.numerical_rank = numerical_ranks[rank]


    def __str__(self):
        return self.rank + " of " + self.suit

### Deck Class 

This class defines the blueprint of a single deck.

In [37]:
class Deck:

    #Initialize all cards in a deck
    def __init__(self):
        self.all_cards = []
        
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit, rank))

    def add_cards_to_deck(self, cards):  
        if type(cards) == type([]):                #Add multiple cards to a deck
            self.all_cards.extend(cards)
        else:                                      #Add a single card to a deck
            self.all_cards.append(cards)

    def draw_cards_from_deck(self, number_of_cards = 1):
        drawn_cards = []
        
        for i in range(number_of_cards):
            drawn_cards.append(self.all_cards.pop(0))                #Draw a configurable number of cards from the deck
            
        return drawn_cards

    def shuffle_deck(self):
        return random.shuffle(self.all_cards)

    def reinstantiate(self):
        self.__init__()

    def __str__(self):
        deck_str = ""
        
        for index, card in enumerate(self.all_cards):
            deck_str = deck_str + str(index + 1) + " - " + str(card) + "\n"

        return deck_str

    def __len__(self):
        return len(self.all_cards)

### Hand Class 

This class defines the blueprint of a single hand.

In [39]:
class Hand:

    def __init__(self, is_dealer = False):
        self.hand_strength = 0
        self.blackjack_deck = Deck()
        self.blackjack_deck.shuffle_deck()
        
        if is_dealer:
            self.face_up_cards = self.blackjack_deck.draw_cards_from_deck()
            self.face_down_cards = self.blackjack_deck.draw_cards_from_deck()
        else:
            self.face_up_cards = self.blackjack_deck.draw_cards_from_deck(2)
            self.face_down_cards = []

        for card in self.face_up_cards:
            self.hand_strength += card.numerical_rank
            
            if card.rank == "Ace" and self.hand_strength > 21:
                self.hand_strength -= 10
                card.numerical_rank = 1

    def add_card_to_hand(self):
        drawn_card = self.blackjack_deck.draw_cards_from_deck()            
        self.hand_strength += drawn_card[0].numerical_rank
        self.face_up_cards.extend(drawn_card)

        for card in self.face_up_cards:
            if self.hand_strength > 21 and card.rank == "Ace" and card.numerical_rank == 11:
                self.hand_strength -= 10
                card.numerical_rank = 1

    def __str__(self):
        hand_str = ""
        hand_str_list = []
        
        for index, card in enumerate(self.face_up_cards):
            hand_str_list.append(str(card.numerical_rank) + " - " + str(card))

        hand_str = '\r\n'.join(hand_str_list)
        
        return f"Hand Strength : {self.hand_strength}\n{hand_str}"

    def __len__(self):
        return self.hand_strength

### Player Class 

This class defines the blueprint of a single player.

In [41]:
class Player:

    def __init__(self, name, balance):
        self.name = name
        self.hand = Hand()
        self.remaining_balance = balance
        self.stayed = False
        self.bet_amount = 0

    def __str__(self):
        return "Name: " + self.name + "\nBalance: " + str(self.remaining_balance) + "\n\n" + str(self.hand)
        
    def hit(self):
        self.hand.add_card_to_hand()
        
    def stay(self):
        self.stayed = True

    def bet(self, amount):
        self.bet_amount = amount
        
        if amount % 10 != 0:
            print("ERROR- Betting amount must be a multiple of 10")
            return False
        elif self.remaining_balance - amount < 0:
            print("ERROR- Insufficient balance")
            return False
        else: 
            print(f"\n******************** A OF {self.bet_amount} HAS BEEN PLACED SUCCESSFULLY ********************")
            self.remaining_balance -= amount
            return True

    def update_balance(self, amount):
        self.remaining_balance += amount

    def get_hand_strength(self):
        return self.hand.hand_strength

    def reinstantiate_hand(self):
        self.hand = Hand()
        self.bet_amount = 0
        self.stayed = False

In [42]:
class Dealer(Player):

    def __init__(self):
        Player.__init__(self, name = "Dealer", balance = 10000000)
        self.hand = Hand(is_dealer = True)
        
    def __str__(self):
        if len(self.hand.face_down_cards) == 0:
            return str(self.hand)
        else:
            return str(self.hand) + "\nXXXXX HIDDEN CARD XXXXX"

    def unhide_cards(self):
        self.hand.face_up_cards.extend(self.hand.face_down_cards)
        
        for card in self.hand.face_down_cards:
            self.hand.hand_strength += card.numerical_rank
            
        self.hand.face_down_cards = []

    def reinstantiate_hand(self):
        self.hand = Hand(is_dealer = True)
        self.bet_amount = 0
        self.stayed = False

## THE GAME BEGINS

### SETUP

In [45]:
def show_hand(player):
    print(f"\n******************************\n{player.name.upper()}'s CARDS\n******************************")
    print(player)
    print(f"******************************\n")

def do_player_bust(dealer, player, all_players):
    dealer.unhide_cards()
    #clear_output()
    
    print(f"\n##############################\n{player.name.upper()} BUSTED \n##############################\n")

    for p in all_players:
        if p == player:
            dealer.update_balance(p.bet_amount)

        show_hand(p)

    show_hand(dealer)
    
    print("\n##############################\n")

def do_player_blackjack(dealer, player, all_players):
    dealer.unhide_cards()
    #clear_output()
    
    print(f"\n##############################\n {player.name.upper()} HAS A BLACKJACK \n##############################\n")

    for p in all_players:
        if p == player:
            p.update_balance(p.bet_amount * 2.5)
            dealer.update_balance(-(p.bet_amount * 1.5))
        
        show_hand(p)

    show_hand(dealer)

    print("\n##############################\n")

def do_player_win(dealer, player, all_players):
    dealer.unhide_cards()
    #clear_output()
    
    print(f"\n##############################\n {player.name.upper()} WINS \n##############################\n")

    for p in all_players:
        if p == player:
            p.update_balance(p.bet_amount * 2)
            dealer.update_balance(-(p.bet_amount))
        else:
            dealer.update_balance(p.bet_amount)
            
        show_hand(p)

    show_hand(dealer)
    
    print("\n##############################\n")

def do_dealer_bust(dealer, all_players):
    #clear_output()
    
    print("\n##############################\n DEALER BUSTED \n##############################\n")
    
    for p in all_players:
        
        if p.stayed:
            dealer.update_balance(-p.bet_amount)
            p.update_balance(p.bet_amount * 2)
            
        show_hand(p)

    show_hand(dealer)

    print("\n##############################\n")

def do_dealer_win(dealer, all_players):
    #clear_output()
           
    print("\n############################## \nDEALER WINS \n##############################\n")

    for p in all_players:
        
        if p.stayed:
            dealer.update_balance(p.bet_amount)
            
        show_hand(p)

    show_hand(dealer)
    
    print("\n##############################\n")

def do_draw(dealer, all_players):
    #clear_output()

    print("\n##############################\n DRAW \n##############################\n")

    for p in all_players:
        p.update_balance(p.bet_amount)
        show_hand(p)

    show_hand(dealer)

    print("\n##############################\n")

In [46]:
#Create the deck
#blackjack_deck = Deck()
#blackjack_deck.shuffle_deck()

In [47]:
print("**********WELCOME TO BLACK JACK!!!**********\n")

all_players = []
number_of_players = 0

while number_of_players == 0:
    number_of_players = int(input("Enter number of players: "))
    
    if number_of_players > 0 and number_of_players <= 4:
        
        for i in range(number_of_players):
        
            name = input(f"Please enter your name for player {i + 1}: ")
            balance = 0
            
            while balance == 0:
                balance = int(input("Please enter your balance: "))
        
                if int(balance) < 0:
                    print("Invalid Value, balance cannot be a negative number")
                    balance = 0
                elif balance < 1000:
                    print("Minimum balance must be 1000")
                    balance = 0
            
            #Create the player
            new_player = Player(name, balance)
            all_players.append(new_player)
                           
        break
        
    else:    
        print("The game can only have between 1 and 4 players")
        number_of_players = 0

#Create the Dealer
dealer = Dealer()

**********WELCOME TO BLACK JACK!!!**********



Enter number of players:  1
Please enter your name for player 1:  Sahil
Please enter your balance:  5000


In [48]:
round_over = False

while not round_over:

    top_players = []

    clear_output()
    show_hand(dealer)

    #blackjack_deck.reinstantiate()
    #blackjack_deck.shuffle_deck()
    
    for player in all_players:
        bet_successful = False

        print(f"******************************\n{player.name.upper()}'s TURN\n******************************")
        print(f"Name: {player.name}")
        print(f"Balance: {player.remaining_balance}")
        print(f"******************************\n")
    
        while not bet_successful:
            amount = int(input(f"{player.name}, how much would you like to bet? "))
            bet_successful = player.bet(amount)
    
        if player.get_hand_strength() > 21:
            do_player_bust(dealer, player, all_players)
            
        elif player.get_hand_strength() == 21:
            do_player_blackjack(dealer, player, all_players)
            
        else:
            while not player.stayed:
                #clear_output()
                show_hand(player)
                
                action = input(f"{player.name}, do you want to hit or stay? (H/S)")
            
                if action.upper() == "H":
                    player.hit()
            
                    if player.get_hand_strength() > 21:
                        do_player_bust(dealer, player, all_players)
                        break
            
                    elif player.get_hand_strength() == 21:
                        do_player_blackjack(dealer, player, all_players)
                        break
            
                elif action.upper() == "S":
                    player.stay()
                    
                    #clear_output()
                    #show_hand(player)

                    if len(top_players) == 0:
                        top_players.append(player)
                    elif player.get_hand_strength() > top_players[-1].get_hand_strength():
                        top_players.clear()
                        top_players.append(player)
                    elif player.get_hand_strength() == top_players[-1].get_hand_strength():
                        top_players.append(player)
                    
                    break
            
                else:
                    #clear_output()
                    #show_hand(player)
                    print("Invalid Action, please try again")

    active_players = [player for player in all_players if player.stayed]

    if len(active_players) != 0:
        dealer.unhide_cards()
    
        if dealer.get_hand_strength() > 21:
            do_dealer_bust(dealer, all_players)
        
        elif dealer.get_hand_strength() == 21:
            do_dealer_win(dealer, all_players)
    
        else:
            while dealer.get_hand_strength() <= 16:
                dealer.hit()

            if dealer.get_hand_strength() > 21:
                do_dealer_bust(dealer, all_players)
            elif dealer.get_hand_strength() == 21:
                do_dealer_win(dealer, all_players)
            elif dealer.get_hand_strength() < top_players[0].get_hand_strength():
                if len(top_players) == 1:
                    do_player_win(dealer, top_players[0], all_players)
                else:
                    do_draw(dealer, all_players)
            elif dealer.get_hand_strength() == top_players[0].get_hand_strength():
                do_draw(dealer, all_players)
            elif dealer.get_hand_strength() > top_players[0].get_hand_strength():
                do_dealer_win(dealer, all_players)
    
    dealer.reinstantiate_hand()
    
    for player in all_players:
        continue_response = ""
        
        while continue_response.upper() not in ["Y", "N"]:
            continue_response = input(f"{player.name}, do you want to continue playing? (Y/N) ")
    
            if continue_response.upper() not in ["Y", "N"]:
                print("Invalid response, please try again")
                continue_response = ""
            else:
                if continue_response.upper() == "Y":
                    player.reinstantiate_hand()
                    round_over = False
                else:
                    round_over = True
    
#while True:
#    pass


******************************
DEALER's CARDS
******************************
Hand Strength : 10
10 - Jack of Spades
XXXXX HIDDEN CARD XXXXX
******************************

******************************
SAHIL's TURN
******************************
Name: Sahil
Balance: 4500
******************************



Sahil, how much would you like to bet?  500



******************** A OF 500 HAS BEEN PLACED SUCCESSFULLY ********************

******************************
SAHIL's CARDS
******************************
Name: Sahil
Balance: 4000

Hand Strength : 10
4 - Four of Diamonds
6 - Six of Hearts
******************************



Sahil, do you want to hit or stay? (H/S) h



******************************
SAHIL's CARDS
******************************
Name: Sahil
Balance: 4000

Hand Strength : 18
4 - Four of Diamonds
6 - Six of Hearts
8 - Eight of Hearts
******************************



Sahil, do you want to hit or stay? (H/S) s



############################## 
DEALER WINS 
##############################


******************************
SAHIL's CARDS
******************************
Name: Sahil
Balance: 4000

Hand Strength : 18
4 - Four of Diamonds
6 - Six of Hearts
8 - Eight of Hearts
******************************


******************************
DEALER's CARDS
******************************
Hand Strength : 19
10 - Jack of Spades
2 - Two of Clubs
7 - Seven of Diamonds
******************************


##############################



Sahil, do you want to continue playing? (Y/N)  n
